In [0]:
%%sh
pip install -q dash
pip install -q dash_core_components
pip install -q dash_html_components
pip install -q dash_table

In [0]:
%%sh
# get ngrok
curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.1M  100 13.1M    0     0  37.6M      0 --:--:-- --:--:-- --:--:-- 37.6M


In [2]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
data = pd.read_csv('https://raw.githubusercontent.com/sachinsirohi007/Machine-Learning-Algorithms/master/Algorithms-Part1/Project/diabetes.csv', sep=',')
X = data.values
y = data['Outcome'].values
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.30)
DecisionTree = tree.DecisionTreeClassifier(max_depth=10)
DecisionTree.fit(X_train, y_train)
joblib.dump(DecisionTree, 'classifier.pkl')


['classifier.pkl']

In [0]:
%%writefile dash_app.py
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from collections import Counter
import unicodedata
import pandas as pd
from random import randint
import threading
import pandas as pd
import os
import base64
from sklearn.externals import joblib

app = dash.Dash()
im1='https://images.vexels.com/media/users/3/145502/isolated/preview/9824e521b3d8c5e4f893b269cf6f9128-breast-cancer-ribbon-by-vexels.png'
im2='https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4oO4Jb23asIfgGBx52Nb37qaXrDeH7WZ8BUkIcfgWYXTp08Cj'
im3='https://images.vexels.com/media/users/3/145502/isolated/preview/9824e521b3d8c5e4f893b269cf6f9128-breast-cancer-ribbon-by-vexels.png'
im4='https://upload.wikimedia.org/wikipedia/en/c/cd/Anaconda_Logo.png'
im5='https://seeklogo.com/images/S/spyder-logo-68D7CF8B2C-seeklogo.com.png'
im6='data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxATEBUTEw8VFRQXFxUXFRUXFQ8PFRYSFREWFhUYGBUYHSogGBslGxUVITEhJSkrLi4uFx8zODMtNygtLisBCgoKBQUFDgUFDisZExkrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrKysrK//AABEIAOEA4QMBIgACEQEDEQH/xAAcAAEBAAMAAwEAAAAAAAAAAAAAAQYHCAIEBQP/xABPEAABAgQDBQMGBwsJCQAAAAABAAIDBBExIWFxBQcSE0EGUbEIFCKBkfEVI3OTs9HTJCUyM1JTVHJ0gpI1VWJjlKGytNIWGDRCQ0RFg6L/xAAUAQEAAAAAAAAAAAAAAAAAAAAA/8QAFBEBAAAAAAAAAAAAAAAAAAAAAP/aAAwDAQACEQMRAD8A3ele5D3KZD3IKT0CE9OqlsBdLaoKTTVCaKW1S2JugtaXSvUqZlMygoPUoD7FL6JfTxQUGuiVropfRMggte5CegUyHuS2AQUnoEJ9qltUtmUFJpqlaXUtibpmUFr1KA9SpmUvibIKCgNdFL6eKX0QUGuiV7lL4BMggpPQIT0ClsAltUFJ9qtV42zKoFL3QVVRVB4k9ApbAXVJ7rqW1QLapbVLapbE3QLYm6Zn3JmfcmZQMyl9Evol9PFAvp4pfRL6JkEDIJkPcmQ9yWwCBbAJbVLapbVAtmUtibpbE3TMoGZTMpmUvibIF8TZL6eKX08UvogX0S+AS+ATIIGQ9yWwCWwCW1QLapbMpbMpbE3QLYm6oHUqZlUDqUFVUqqg8SaaqW1VJopbE3QLYm6Zn3JmfcmZQB3lL6JfRL6eKBfTxS+iX0TIIGQXoz+25SA4MizcCE4itIkWFCNO+jiCvfyC437YR3xNoTT3uLnGPGqSanCI4AaAAADoAg6/kZ+DGbxQI0OK2tOKG9kVtetS0kVXsW1XOnk6zLxtOKwOPC6XeXN6FzIkPhJHeOJ38R71vTthMPhbOnIjHFsRstMOa4XD2wHlpGhAKD9JjtHIQ3Fj56Xa8GjmujwGOB7iC6oX0JeOx7Q9r2va4Va5pDmkdOEjAhcRnFdBeThMOMlMtLiWsjN4QTUN4ofpU7q0CDbpIALnGlMcgF8lnafZ5dT4QlSa0DRMQCa6cV1im/aO9uxonC4t4okJpoaVYXVIOtAuYUHb4xx6L8puahw2F8SI2HDF3Pc1jfW44ALEdzsw+JsSUdEcXENiNqb8MOYiMYNA1rR6lgflKzT/ALihhx4Dz3FvQvby2g06kBzh+8UG3ZXtBJRniHCnZd7jZrI0F7jo1rqr6WQXEcKI5rg5ri1wILSCWkOBqCCLEFdrSUQuhMPUsaScy0EoP2yHuS2AS2AS2qBbVLZlLZlLYm6BbE3TMpmUzKBmVRjipfE2VGOnig8qoiIPE4YqZlU95UzKBmUvol9Evp4oF9PFL6JfRMggZBMh7kyHuX5zUyyExz3vaxjQXOc4hrWtAqXOJsEH62wC4y7Tj7umvl430rlv+c35bJhvLWMmYoH/ADshw2tdpzHtd7QFzztiaEaYjRWghr4sR4BoCA95cK064oNh+Tyfvs/9mi/SwlvTt0PvVPVv5pM/5d65u3V9rIGzZ50xGZEewwXwwIYY53E57HA0c4Cnonqtj9o99OzpiTmYDZeaD4sCNCaXMlw0OiQnMBJEUmlSK4INDLfvk2AeazfysP6MrQS2bul3hymzIMeHMQozzEe1zeU2E4ANaQa8T24oNmb/AA/eZ3y0GntK5mW39529SS2jIGWgwZhjzEhuBiNgtbRpNcWxCf7lqBB1PuTFdhSnd8f/AJuMsD8pYjmSP6sx4wl+e73e3IyOzYEpFgzLnw+bxOY2CWnjjxIgoXRAbPHRY3vd7cyu1HSxgQozBCEUO5rYba8ZZSnC935JQa8Xa2zT8RCA/Ns/wBcUronZm/HZYZDY6DNNo1rS8w4JaCGgEnhiE00CDa1tUtmV6eyNqQJmCyPAiiLDiCrXCuPQgg4tIOBBxFMV7lsTdAtibpmUzKZlAzKXxNkvibJfTxQL6eKta6KX08Va9yDyRSiqDxI6lS+ipHsUvp4oF9PFL6JfRMggZBU9wXobfnXQJSPFYAXQoMWIK24mQ3OFfWFyLMdqdoRHue+emC5xJJ50UYnuANAMhgEHZNsAtY+ULNPh7JaxpoIsxDY/NgZEiU/ihsPqWgB2hnv02Y+ejf6l+E5tSYijhizEWI0GoD4kSIAaEVo43xPtQemiIgIiICIiAiIgIiICIiDe3k1TbjDnYRPoNdAe0dz4jYjXn1iGz2LdQHUriiS2hHhV5MaJDrTi4Hvh1pWleE43PtXtHtDPfpsx89G/1IOzAOpS+JsuNB2jngQRPTNRYiPHFDl6S6n3b7VizeypWPGNYjmEPNuJzIjmcRzPCCcygyS+nil9PFL6eKXwFkC+AsrXoFMgrkEFoqoqg8SK6KX0VOOimQQMgmQ9yZD3JbAIPX2lJtjQYkB1eGIx8N1L8L2lpNe+hXPk3uJ2mHkMjyz219FxfFhkt6Et4DwnKpXRdtV8iZ7U7OhPcyJtCVZEaaOa6YgMcD3FpdUINDO3GbW/OSvzsX7NfC7Z7t57ZsBseYfBLHRBDAhve93EWOcKgsGFGFdIjtlsq/wpJ1/aZb/Uta7+9vScxs6E2BOQIzxMscWQo0GM4M5EYEkNJNKkY5hBoZERARFlO7Ps/wCe7Ul4JFWB3Mi9RyofpEHJxAb+8gxZF2idiyhtKwfmoX1LEd6vZKBH2XHEGXhtiwhzmFjGtNYdS5vojGrOMU7yEHLiIvrdkmB20JQEAgzMAEEAggxm1BHUIPkou0TsWUH/AGkGvyUL6lqbyh9nwYcpLGHBhsJjOqWMYzDlnDAWQaHREQZR2K7DTe0+b5u6E3k8HHzHPZXmcfDThaa/gH+5ZONxe1qfjJX52L9mvpeT3teUlzO+cTUGDxeb8PNiw4PFw86tOMitKj2hbi/2y2Uf/KSdP2mWx/8ApBo2FuL2qSKxZUDv5kY0HfTlre/ZLYbZKSgyrXlwhNoX0pxOLi55A6AuccF4N7YbMcQ0bTlDUgACZlySTYAcS+1Wtrd/1IF8BZMgmQTIIGQVGGHVS2AuqMNUHkiiqDxPcpkPcqT0ClsAgWwCW1S2qW1QfM7UR3Q5CaexxD2y8dzXC4c2C4gjQhcZ3XbkxAa5jmvaHNcC1zTiC1woW07iCtSzW4SSLnFs5GY0kkM4Yb+Ed3FhVBz6i38NwMrT/j43zcL61h+87djB2ZKMmIczEiF8ZsLhc1jQAYcR9aj9Qe1BrFERAW+vJz2BwwI865uMR3Jhn+rZ6UQjIuLR/wCtaIgQXPc1rQXOcQ1rRiS4mgHtXY/ZXY7ZOSgSrKfFQ2tcRYvOMR3rcXH1oPqPeB1AsKnDEmgGpJA9argKUpXvrjhmtY79O07pOBKshn4x0wyMR3slnNiUOsTl+wrZElNMiQmRIZ4mxGte03q17Q5pr3UIQcidutg+ZbRmJahDGPJh3/FP9KHib+iQNQV+HY/+UZP9pl/p2LbHlHbAxl51o74EU54vhGnzgr+qtT9jz98ZP9pl/p2IOybYm60/5SIPmcr8s76IrcGZWn/KRr5nK/LO+iKDn9ERARZ9up7AQtqmY5kd8Lk8qnC1ruLmcyta93APas/G4KV/T43zcP60Ggl1hujmHv2LJlziTwObU9GsjPY0eprQPUsNbuClK4z8YjrRkIYa9Ctq7I2bClYEOXgN4YcNoa0Xw6knqSak5lB7mQS2AulsBdLZlAtmVQKXupbE3VA6lBVVFUHiT0CltVSfapbVAtqlsTdLYm6ZlAJABc4gU78AAtfTO+XYzXlvOiP4SRVsJxaadQTSuqyvtc1x2fN0Br5tMBoGJqYD6etcbIOmTvq2N+XG05J+tYLvh3hyG0ZKHBlnRC9sdsQh0MsHCIUVpxr3vC1AiAiIg2BuQ2AZrasN7hWHLjnOqKjjBpCGvGQ79wrp/ILWW4LYPI2aY7m/GTL+K1KQWVbDHt43aOC2JtOehy8GJGiOoyGxz3nE0a1pJNBe1kGvN5G7GNtObbG8+bDa2G1jIZhufShLnOJ4hiS7u6BZp2Q2TEk5GDLRIoiuhN4A8NLKsDjwChJpRtB6liw3zbF/PxPmYv1L63ZjeJs2ejmDLxXOi8JfR0N8OrWkAgE9cbaoPc7fbB892bMS9KvcwmH8sz04Yy9IAaErlbsefvjJ/tMv9OxdkZlc09rNh+adp4TQ2jIs1Lx4f6sWO0u0o8PHqQdLU6lag8pE/ccr8s76IrcFK3stP+UifuOV+Wd9EUHP6IiDZe5jtpJ7N8685c8c3kcHAwxPxfN4q44fhhbOO+vY3R8b5k/WuZkQdNs31bGwHMjDMwXUGZpis+kpuHFhsiQnh7HtDmOGIc1wqCuJl1bueBGw5OoNeGJQG9DHiEeqhCDMrZlLYm6WxN0zKBmVQOpUzKoxxQWqqlVUHiTTVS2JuqcMVMygZlMymZS+JsgUreyw2Z3V7EixHRHbPaC41PDEmYTa5MY8NaMgAstm5lkOG+I88MNjXPcf6LWlzjpQFaVm/KA9MiHs2rK+iXR+Fxb0qAwgHKp1QZuN0ewjaQw+XnftFgG+rsPs2RkYUWUleW90w1jnc2YiVYYMVxFHvIu1uOS8z5QMSn8mN/tDvs1i+8Peg7akqyXMmIIZFbF4hFMWvDDeylOAfl1rkg14iIg3Fs3foYMGHBh7LYGQ2NY0c934LGho/wCn3BfM7Y74Ys9JRJUSYgiJw8UQRXPPC14cRThF+EC9qrWCIC+z2Q2++RnYU0xnGYZNWEloc1zC1wJ6YOPRfGRBur/eAiV/kxnz7vs1ivbDeX59MykwZFsN8rEDxSKX8xoex/AfQFMWX/pFa/RBup3lARP5sZ8+77NYlvG3lP2pBhQ3SjYPLeX1EQxa1bw0pwiiwJEBERBtLcd2SkdoGb87l+by+Ry/jI0Lh4+dxfi3CteFt+5bVO6PYX6Bj8vO/aLRu7beA7ZXPIlRG53KvEMLh5fM7mmteZ/cs2b5QET+bG/2h32aDPmbpdhNIPmFSDUVjTbhUd4MShGqzOBBZDY1rWhrWgNa1oDWtaBRrWgWAHRaPh+UA+uOzG060mDWmXxa3H2e2xCm5WFNQ68EVvE0HAtxo5pzBBB0QfQzKZlMyl8TZAvibKjHRS+nirWunig8kREHie8qZlUjqVL4myBfE2S+nil9PFL6IPndpZZ8aSmYUMVc+BGY0Wq90JzWj2kLjOJDc1xa5pa5pILSC0hwNCCDYg9F27kF+MWUhOOMJhPUlrXH2kIOJkXa52fAsIEP+Bn1LWPlCbPaNlw3MhNHDMwy8ta0ENMKM2pIFuItHrCDnZERAREQEREBERAREQEREBFu7yb5Jrmzz3wg5tZdrS5od6QEYuArk5tdQt0/B8C5gw/4GfUg4oK603VyESBseUZFYWPDHOLTgQHxXvbUdDwuGHRZIyRgg8XKYCMRRjAR66XX73xNkC+Jsl9PFL6eKX08UC+nirXuspfAWVr0CDyoilFUHiQpfTxVIropfRAvol8Al8AmQQMh7ktgEtgEtqgW1XqbX2XBmID4EeGIkOIOF7T1xqCD0IIBBFiAV7dsylsTdBpae8n+GXkwtpOYzo18BsVw/fa9oP8ACFpLacpyo8WDxcXLiPZxUpXgeW1p0su1wOpXGXac/d0z8vG+lcg+ru87I/Cc26X845NITonHwc38FzG04eJv5d69FnO29xpl5SPMfCXFyYUWLw+b8PFy4bn04uaaVpStCvm+TwB8LP8A2aL9LCW9O3eOyp7u80mfX9zvQceLPt227U7VhRYnnnI5T2tpyudxcTa1rxtosBW/fJtH3LN/Kw/oygw3t7un+DZMzPn3Oo9jODkcr8MnHi5htTuWtF0zv9P3mcP66D4lczINpditzx2hIwpv4Q5fM5nxfI5nDwRnw/wuYK14K26r4e8vsD8FOgAzXP5oiH8VyeHgLB+W6tePKy3juUP3ilO/4/8AzcZYH5SzfjJH9WY8YSDSi3dIbhGuax79pnhcGuLWy4a7EA0DjEPtotIrtfZn4iGT+bZ/gCD0ey/ZyW2fLNgQGcLBUkk8T3vNKveerjQewAUAAX1symZS+JsgXxNkvp4pfTxS+nigX08UvgLJfAWTIIGQVyCmQVtggqqiqDxIropfAKnuUyCBkPclsAlsAltUC2qWzKWzKWxN0C2JumZTMpmUADqVxp2qBE/NAih84jgjqDznYLsu+Jssb2z2C2VNxTGjyTHxDd4MWEXUwq7gcKnM44INK+TvDJ2rENDQS0SvcKxYVKrenbdpdsydDQTWVmQABUk8h9AAvPs92YkpJrmyssyEHU4yOJznUsC9xLiBU4V6lfXOOHTr9SDiBb+8m1p80mjTAxmCvTCHj68R7VmU1uw2LEeXHZ0OpNTwujw2jRrHBo0AWR7K2XAlYTYEvBbChts1ooMbkm5J7zigwff237zPoK0iwSchxEVPrI9q5kXbM7KQ4sN0KJDbEY8UcxwDmuB7wVig3WbDBB+D2E1rTmTBFdC+lMkH5blmluwpSrSCecaHA0M1FIOlCD61gXlLQzxyJ6UmBXpWsI0W8IEFkNjWtaGtaA1rWgNa1oFA1rRgAB0Xpbc2HKzcLlzUBsVlahrq+i61WuGLTiRUEXQcYFdsbOYRBh8WBDGCh6HhFfWsc2du32PAiNiskIYe01aXOjRQCLHhe4io76YLKr4myBfE2S+nil9PFL6eKBfTxS+Asl8BZMggZBMgmQS2AugWwF1RhqpbMqjDVBVVFUHiT0ClsAqT0CltUC2qWzKWzKWxN0C2JumZTMpmUDMpfE2S+Jsl9PFAvp4pfTxS+nil8BZAvgLJkEyCZBAyCWwF0tgLpbMoFsylsTdLYm6ZlAzKZlMyl8TZAvibJfTxS+nil9PFAvp4pfAWS+AsmQQMgmQTIJbAXQLYC6WzKWzKW1QLaqgdSpbEqgdSgqqIg8SfapbVeRUApj1QS2JumZVA6lAOpQTMpfE2VpW6Urp4oJfTxS+niqcdEPcgl8BZMgqe4JkEEyCWwF1bWulKZlBLZlLYm6oFMeqAdSgmZTMqgdSlK3QS+Jsl9PFWldEOOnigl9PFL4Cyp7uiHuCCZBMgrkEtZBLYC6WzKtKZlAKaoJbVLYlUDqUA6lBMz7lRjiUpXEpfRBaqoiCIqiCIVUQCiIgKBVEECKogKKogiKoghQqogIiIAUCqIIiqIIiqIIqiIIVURBEREH//2Q=='
im7= 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/37/Plotly-logo-01-square.png/220px-Plotly-logo-01-square.png'
im8='https://cdn0.iconfinder.com/data/icons/trending-tech/94/artificial_intelligence-512.png'
im9 ='https://cdn-images-1.medium.com/max/1200/1*lkqc68a6b7_TLALs5fmI6A.png'
clf=joblib.load('classifier.pkl')

        

app.layout = html.Div([
    html.H1(children=' Intilegent Diabetic Prediction System ', style={'marginBottom': '12px'}),
    html.Div(children = [html.Img(src=im1,style={'width': '20%', 'marginLeft': 'auto', 'marginRight': 'auto'}),
                         html.Img(src=im2,style={'width': '20%', 'marginLeft': 'auto', 'marginRight': 'auto'}),
                         html.Img(src=im3,style={'width': '20%', 'marginLeft': 'auto', 'marginRight': 'auto'})]),
    html.H2(" Please observed Value  ",style={'marginBottom': '6px'}),
    html.Div(children=[dcc.Input(id = 'Temperature',placeholder='  Pregnancies ',type='text',value=''),
                       dcc.Input(id = 'Pressure',placeholder='  Glucose ',type='text',value=''),
                       dcc.Input(id = 'Humidity',placeholder='  BloodPressure ',type='text',value=''),
                       dcc.Input(id = 'Cloud',placeholder=' SkinThickness ',type='text',value=''),
                       dcc.Input(id = 'Windspeed',placeholder='  Insulin ',type='text',value=''),
                       dcc.Input(id = 'WindDirection',placeholder='  BMI',type='text',value=''),
                       dcc.Input(id = 'DiabetesPedigreeFunction',placeholder='  DiabetesPedigreeFunction',type='text',value=''),
                       dcc.Input(id = 'Age',placeholder='  Age ',type='text',value='')]),
    
    html.Button(id='submit-button', n_clicks=0, children='Sumit', 
    style={'marginTop': 15, 'marginBottom': 25}),
    html.Div(id='my-div')], style={'textAlign': 'center'})

app.css.append_css({"external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"})


@app.callback(
    Output(component_id='my-div', component_property='children'),
    [Input(component_id='submit-button', component_property='n_clicks'),
     Input(component_id='Temperature', component_property='value'),
     Input(component_id='Pressure', component_property='value'),
     Input(component_id='Humidity', component_property='value'),
     Input(component_id='Cloud', component_property='value'),
     Input(component_id='Windspeed', component_property='value'),
     Input(component_id='WindDirection', component_property='value'),
     Input(component_id='DiabetesPedigreeFunction', component_property='value'),
     Input(component_id='Age', component_property='value')]
)
def update_output_div(n_clicks,x1,x2,x3,x4,x5,x6,x7,x8):
    if n_clicks > 0:
        val = []
        val.append(x1)
        val.append(x2)
        val.append(x3)
        val.append(x4)
        val.append(x5)
        val.append(x6)
        val.append(x7)
        val.append(x8)
        val.append(x1)
        out = clf.predict([val])[0]
        if out ==1:
            return html.H2("Diabetic Predicted")
        else:
            return html.H2("Diabetic Not Predicted")
if __name__ == '__main__':
    app.run_server()
        

Overwriting dash_app.py


In [0]:
# launch ngrok
get_ipython().system_raw('./ngrok http 8050 &')

In [0]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://f931cfa2.ngrok.io


In [0]:
!python dash_app.py

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.

 * Serving Flask app "dash_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
/usr/local/lib/python3.6/dist-packages/dash/resources.py:68: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plot.com/external-resources

127.0.0.1 - - 